<a href="https://colab.research.google.com/github/veroorli/ProjetProg/blob/master/TME722.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Querying nested data 

This lab session complements the previous session with several concepts:
* manipulating irregular, nested data represented in JSON format
* using windowing functions, multi-dimensional aggregations, and pivot tables

## Questions

The schema indicates each attribute with nullable=true which is not very informative since we can not know whether a field has missing values.
The following instructions count the number of rows w/o missing values for some specific fields of interest.

In [ ]:
att_list = ["event", 
                 "event.event_id", 
                 "event.event_id.post_id", 
                 "event.event_id.post_owner_id", 
                 "event.event_id.comment_id", 
                 "event.event_id.shared_post_id", 
                 "event.author", 
                 "event.attachments", 
                 "event.geo", 
                 "event.tags",
                 "event.creation_time"]

In [ ]:
for att in att_list:
    print("count of %s=%s"%(att, data.where(att+ " is not null").count()))

#### Check that `event.event_id.comment_id` is present only when `event.event_type='comment'`

In [ ]:
data.filter((col('event.event_type') == 'comment')
 & (col("event.event_id.comment_id").isNull())).count() == 0


#### How many distinct post ids are there?

In [ ]:
nb_posts = data.select(countDistinct(col('event.event_id.post_id'))
.alias("nb_distinct_posts"))
nb_posts.show()

#### How many distinct post_ids per event type

In [ ]:
post_per_type = data.groupBy(col('event.event_type'))\
.agg(countDistinct(col('event.event_id.post_id')).alias("nb_distinct_posts"))
post_per_type.show()


#### Flattening lists of tags

In `data`, each object is associated with an array of tags accessed from `event.tags` (see the schema). Write an instruction to add a `tag` column containing a single tag obtained by flattening the `tags` array

In [ ]:
data_with_tags = data.withColumn('tag', explode(col('event.tags')))
data_with_tags.show()

#### Return the number of distinct post_id per tag. Sort in descending order of count

In [ ]:
from pyspark.sql.functions import desc

objects_per_tag =data_with_tags.groupby(col('tag'))\
                .agg(countDistinct(col('event.event_id.post_id'))\
                .alias("nb_posts"))\
                .orderBy(desc(col('nb_posts')))

objects_per_tag.show()



#### Return the number of distinct author.id per tag. Sort in descending order of count

In [ ]:
authors_per_tag = data_with_tags.groupby(col('tag'))\
                  .agg(count_distinct(col('event.author.id'))\
                  .alias("nb_authors"))\
                  .orderBy(desc(col('nb_authors')))

authors_per_tag.show()

+-----------+----------+
|        tag|nb_authors|
+-----------+----------+
|      putin|     15673|
|   grudinin|      6207|
|    navalny|      2580|
|    sobchak|      2288|
|zhirinovsky|      1214|
|      titov|       572|
|  yavlinsky|       347|
+-----------+----------+



#### Fact-checking using Wikipedia

Observe that each tag corresponds to a candidate of the RU2018 Elections (Putin, Titov, etc).
We would like to check the relationship between the count of tags per author and the number of votes associated to each candidate.
We collect, from Wikipedia, the number of votes per candidates and make it available though the dataset `votes` defined as follows.

In [ ]:
from pyspark.sql.types import *

schema_votes = StructType([StructField("name", StringType()),\
                     StructField("party", StringType()),\
                     StructField("votes", LongType())])
                     

raw_votes = [("putin", "Independent", 56430712),\
             ("grudinin", "Communist",8659206), \
             ("zhirinovsky","Liberal Democratic Party",4154985),\
             ("sobchak","Civic Initiative",1238031),\
             ("yavlinsky","Yabloko",769644), \
             ("titov","Party of Growth",556801)\
            ]

votes = spark.createDataFrame(raw_votes,schema_votes) 
votes.show()

+-----------+--------------------+--------+
|       name|               party|   votes|
+-----------+--------------------+--------+
|      putin|         Independent|56430712|
|   grudinin|           Communist| 8659206|
|zhirinovsky|Liberal Democrati...| 4154985|
|    sobchak|    Civic Initiative| 1238031|
|  yavlinsky|             Yabloko|  769644|
|      titov|     Party of Growth|  556801|
+-----------+--------------------+--------+



#### Return for each candidate the number of its votes and the number of authors who tagged it. What do you observe?

tab1.join(tab2.select(col('critère1').alias(''),col('')),on='colonne à joindre').select('colonnes')

In [ ]:
votes_count = votes.join(authors_per_tag.select(col('tag')
.alias('name'),col('nb_authors')),on='name')\
.select('name', 'votes', 'nb_authors').orderBy(desc('nb_authors'))
votes_count.show()

+-----------+--------+----------+
|       name|   votes|nb_authors|
+-----------+--------+----------+
|      putin|56430712|     15673|
|   grudinin| 8659206|      6207|
|    sobchak| 1238031|      2288|
|zhirinovsky| 4154985|      1214|
|      titov|  556801|       572|
|  yavlinsky|  769644|       347|
+-----------+--------+----------+



### Window function

Define a window specification for the `votes` and `nb_authors` columns

In [ ]:

windowSpecVotes = Window.orderBy(desc("votes"))
windowSpecCount = Window.orderBy(desc("nb_authors"))

Using the window specifications, augment `votes_count` with two attributes, `votes_rank` and `nbAuths_ranks`, which rank each candidate  based on the number of votes he received  and the number of authors who tagged a post with his name. 

Hint. The window spans the entire data and does not use any partitioning.

In [ ]:
compare = votes_count.withColumn("votes_rank", rank().over(windowSpecVotes))\
.withColumn("nb_authors_rank", rank().over(windowSpecCount))

compare.show()


### Multidimensional aggregation

We want to create a cube with three dimensions: `tag`, `event_type` and `month`. While the first two attributes are already available, the month column must be extracted from the `creation_time` attribute using a built-in function. 

Create a dataset `dataTagMon` obtained by augmenting `data_with_tags` with an attribute `month` containing the month extracted from the `event.creation_time` attribute.

Hint. You will notice that only months 1, 2 and 3 exist in the dataset.

In [ ]:
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import month

In [ ]:
data_tag_mon = data_with_tags\
.withColumn('month', month(from_unixtime('event.creation_time')))

data_tag_mon.select("month").distinct().show()

+-----+
|month|
+-----+
|    1|
|    3|
|    2|
+-----+



#### For each combination of event_type, tag and month, count the nomber of distinct post_ids

In [ ]:
tag_event_month = data_tag_mon.groupBy("event.event_type", "tag", "month")\
.agg(countDistinct('event.event_id.post_id').alias('count_dist_postids'))\
.orderBy(desc("count_dist_postids"))
tag_event_month.orderBy(desc("count_dist_postids")).show(1)

+----------+-----+-----+------------------+
|event_type|  tag|month|count_dist_postids|
+----------+-----+-----+------------------+
|   comment|putin|    2|              4191|
+----------+-----+-----+------------------+
only showing top 1 row



### Pivot table

#### Using the `tag_event_month` table, create a pivot table by reducing the dimensions to month and event type.

In [ ]:
month_event_type = tag_event_month.groupBy('month')\
.pivot('event_type').sum("count_dist_postids")
month_event_type.printSchema()
month_event_type.show(1)

root
 |-- month: integer (nullable = true)
 |-- comment: long (nullable = true)
 |-- post: long (nullable = true)
 |-- share: long (nullable = true)

+-----+-------+----+-----+
|month|comment|post|share|
+-----+-------+----+-----+
|    3|   6424|5328|  207|
+-----+-------+----+-----+
only showing top 1 row



###  Tag co-occurrence matrix

#### create a dataframe indicating for each pair of distinct tags the author that uses both of them.

In [ ]:
# auth_tag = data_with_tags.

tag_co_occur = data_with_tags.select("event", col("tag").alias("otherTag"))\
.crossJoin(data_with_tags.select(col("event").alias("event2"), "tag"))\
.select("event.author.id", col("event2.author.id")\
.alias("id2"), "otherTag", "tag")\
.filter((col("id") == col("id2")) & (col("tag") != col("otherTag")))\
.select(col("id").alias("authorID"), "otherTag", 'tag')

tag_co_occur.show(1)

+----------+--------+-----+
|  authorID|otherTag|  tag|
+----------+--------+-----+
|-163732739| navalny|putin|
+----------+--------+-----+
only showing top 1 row



#### Build the tag co-occurence matrix indicating for each pair of tags the number of authors that use them

In [ ]:
co_occur_mat = tag_co_occur.groupBy("tag")\
.pivot("otherTag")\
.agg(count('tag'))
co_occur_mat.show(1)

+-----+--------+-------+-----+-------+-----+---------+-----------+
|  tag|grudinin|navalny|putin|sobchak|titov|yavlinsky|zhirinovsky|
+-----+--------+-------+-----+-------+-----+---------+-----------+
|titov|     140|     21|  246|    111| null|       94|         90|
+-----+--------+-------+-----+-------+-----+---------+-----------+
only showing top 1 row



## END